# IGRINS starspot measurements with dashboard

Let's spot-check our data with muler and gollum!

In [ ]:
%config InlineBackend.figure_format='retina'

In [ ]:
from muler.igrins import IGRINSSpectrum, IGRINSSpectrumList
from gollum.phoenix import PHOENIXGrid
import glob
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
fns = glob.glob('../../data/IGRINS/GS-2021A-Q-311/**/SDCH*.spec_a0v.fits', recursive=True)

In [ ]:
len(fns)

### Map filenames to object

In [ ]:
fn = fns[0]

In [ ]:
lookup_dict={}
for fn in tqdm(fns):
    spec = IGRINSSpectrum(file=fn)
    lookup_dict[fn]=spec.meta['header']['OBJECT']
    del spec

In [ ]:
df = pd.DataFrame.from_dict(lookup_dict,orient='index').reset_index()
df = df.rename(columns={0:'Object', 'index':'fn'})[['Object', 'fn']]

In [ ]:
df.Object.value_counts()

Yay, we have up to 5 spectra on several of our targets.

In [ ]:
target = 'V* V479 Car'

In [ ]:
mask = df.Object == target

In [ ]:
fns = sorted(df[mask].fn.values)

### Read in each spectrum 
... to an HPFSpectrumList Object

In [ ]:
m = 13

In [ ]:
spectra = []
for fn in tqdm(fns):
    spectra.append(IGRINSSpectrum(file=fn, order=m))

Let's pick one representative spectra order and make a list of those spectra.

In [ ]:
%matplotlib notebook 

In [ ]:
ax = spectra[0].trim_edges().normalize().barycentric_correct().plot(ylo=0, yhi=2)
for spectrum in spectra:
    spec = spectrum.trim_edges().normalize().barycentric_correct()
    ax.step(spec.wavelength, spec.flux, color='k', alpha=0.3)

In [ ]:
from gollum.telluric import TelluricSpectrum
import astropy.units as u

In [ ]:
for spectrum in spectra:
    print(spectrum.estimate_barycorr().to(u.km/u.s))

In [ ]:
data = spectrum.trim_edges().barycentric_correct().normalize()

wl_lo, wl_hi = data.wavelength.value.min(), data.wavelength.value.max()

In [ ]:
tell_spec = TelluricSpectrum(
                    path="default", wl_lo=wl_lo, wl_hi=wl_hi
                )

In [ ]:
ax = spectra[0].trim_edges().normalize().barycentric_correct().plot(ylo=0, yhi=2)
spectra[4].trim_edges().normalize().barycentric_correct().plot(ax=ax)
for spectrum in spectra:
    spec = spectrum.trim_edges().remove_nans().normalize().barycentric_correct()
    ax.step(spec.wavelength, spec.flux, color='k', alpha=0.1)
    
plt.fill_between(tell_spec.wavelength, tell_spec.flux, 1, color='k', alpha=0.1, label='Telluric')

In [ ]:
grid = PHOENIXGrid(teff_range=(4400, 6200), metallicity_range=(-0.5, 0.5), wl_lo=wl_lo, wl_hi=wl_hi)

In [ ]:
grid.show_dashboard(data=data/1.05)

Neat! We have a decent system!  We need to add some more HPF-specific post-processing to the `muler` toolkit.